In [3]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants
import warnings
warnings.filterwarnings('ignore')
# from stable_baselines3 import DQN
# # from stable_baselines import bench, logger
# from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
import tensorflow
from stable_baselines.common.callbacks import CheckpointCallback
# from modules.env import LupusEnv

In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tensorflow.set_random_seed(constants.SEED)

#### The data

In [5]:
# train_df = pd.read_csv('../data/25_jan/train_set_basic.csv')
train_df = pd.read_csv('../data/missingness/0/training_set.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [6]:
train_df.cutaneous_lupus.value_counts()

0    44200
3     3120
1     2246
2      834
Name: cutaneous_lupus, dtype: int64

In [7]:
train_df[(train_df.ana==0) & (train_df.label==1)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label


In [8]:
train_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [9]:
len(train_df)

50400

In [10]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [11]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No lupus',
 'Lupus',
 'Inconclusive diagnosis',
 'ana',
 'fever',
 'leukopenia',
 'thrombocytopenia',
 'auto_immune_hemolysis',
 'delirium',
 'psychosis',
 'seizure',
 'non_scarring_alopecia',
 'oral_ulcers',
 'cutaneous_lupus',
 'pleural_effusion',
 'pericardial_effusion',
 'acute_pericarditis',
 'joint_involvement',
 'proteinuria',
 'anti_cardioliphin_antibodies',
 'anti_β2gp1_antibodies',
 'lupus_anti_coagulant',
 'low_c3',
 'low_c4',
 'anti_dsdna_antibody',
 'anti_smith_antibody']

#### The Model

In [ ]:
# training_env = LupusEnv(X_train, y_train)
# training_env = DummyVecEnv([lambda: training_env])
# # training_env = VecNormalize(training_env, norm_obs=True, norm_reward=False, clip_obs=10.)

# # Define and train the DQN agent
# model = DQN('MlpPolicy', training_env, verbose=1, seed = constants.SEED)
model = utils.create_stable_dqn(X_train, y_train)
checkpoint_callback = CheckpointCallback(save_freq=100000, 
                                         save_path='../models/logs/sb/dueling_ddqn_prioritized_replay', 
                                         name_prefix='dueling_ddqn_pr_basic')

model.learn(total_timesteps=200000000, log_interval=100000, callback=checkpoint_callback)

# Save the trained DQN agent
model.save('../models/22_mar/dueling_ddqn_PR_lupus_diagnosis')
# training_env.close()

Using new DQN
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 97       |
| episodes                | 100000   |
| mean 100 episode reward | -0.9     |
| steps                   | 466779   |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 200000   |
| mean 100 episode reward | -0.7     |
| steps                   | 940480   |
| success rate            | 0.19     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 300000   |
| mean 100 episode reward | -0.7     |
| steps                   | 1420834  |
| success rate            | 0.24     |
-------------------------

--------------------------------------
| % time spent exploring  | 23       |
| episodes                | 2800000  |
| mean 100 episode reward | 0.2      |
| steps                   | 16098166 |
| success rate            | 0.67     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 20       |
| episodes                | 2900000  |
| mean 100 episode reward | 0        |
| steps                   | 16805476 |
| success rate            | 0.58     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 16       |
| episodes                | 3000000  |
| mean 100 episode reward | -0.2     |
| steps                   | 17499524 |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 13       |
| episodes                | 3100000  |
| mean 100 episode reward | -0       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5900000  |
| mean 100 episode reward | 0.4      |
| steps                   | 42798858 |
| success rate            | 0.85     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6000000  |
| mean 100 episode reward | 0.4      |
| steps                   | 43663296 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6100000  |
| mean 100 episode reward | 0.4      |
| steps                   | 44544500 |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6200000  |
| mean 100 episode reward | 0.4      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9000000  |
| mean 100 episode reward | 0.4      |
| steps                   | 68948398 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9100000  |
| mean 100 episode reward | 0.5      |
| steps                   | 69757967 |
| success rate            | 0.86     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9200000  |
| mean 100 episode reward | 0.4      |
| steps                   | 70550104 |
| success rate            | 0.82     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9300000  |
| mean 100 episode reward | 0.5      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 12100000 |
| mean 100 episode reward | 0.4      |
| steps                   | 93020981 |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 12200000 |
| mean 100 episode reward | 0.4      |
| steps                   | 93785510 |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 12300000 |
| mean 100 episode reward | 0.5      |
| steps                   | 94504930 |
| success rate            | 0.85     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 12400000 |
| mean 100 episode reward | 0.4      |
| steps                  

---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 15100000  |
| mean 100 episode reward | 0.5       |
| steps                   | 114705670 |
| success rate            | 0.84      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 15200000  |
| mean 100 episode reward | 0.5       |
| steps                   | 115415330 |
| success rate            | 0.85      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 15300000  |
| mean 100 episode reward | 0.7       |
| steps                   | 116125812 |
| success rate            | 0.92      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 15400000  |
| mean 100 episode reward | 0.5       |


---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 18100000  |
| mean 100 episode reward | 0.5       |
| steps                   | 134868685 |
| success rate            | 0.86      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 18200000  |
| mean 100 episode reward | 0.3       |
| steps                   | 135510454 |
| success rate            | 0.76      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 18300000  |
| mean 100 episode reward | 0.5       |
| steps                   | 136157581 |
| success rate            | 0.86      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 18400000  |
| mean 100 episode reward | 0.4       |


---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 21100000  |
| mean 100 episode reward | 0.4       |
| steps                   | 153595318 |
| success rate            | 0.81      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 21200000  |
| mean 100 episode reward | 0.5       |
| steps                   | 154189784 |
| success rate            | 0.81      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 21300000  |
| mean 100 episode reward | 0.5       |
| steps                   | 154775094 |
| success rate            | 0.8       |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 21400000  |
| mean 100 episode reward | 0.6       |


---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 24100000  |
| mean 100 episode reward | 0.7       |
| steps                   | 170992850 |
| success rate            | 0.9       |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 24200000  |
| mean 100 episode reward | 0.5       |
| steps                   | 171548820 |
| success rate            | 0.85      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 24300000  |
| mean 100 episode reward | 0.5       |
| steps                   | 172112424 |
| success rate            | 0.83      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 24400000  |
| mean 100 episode reward | 0.4       |
